In [1]:
# %pip install gurobipy

# First, import packages
import pandas as pd
import gurobipy as gp
from gurobipy import GRB

In [ ]:
# Sets P and D, respectively
# When we code sets we can be more descriptive in the name
production = ['Baltimore','Cleveland','Little Rock','Birmingham','Charleston']
distribution = ['Columbia','Indianapolis','Lexington','Nashville','Richmond','St. Louis']



In [ ]:
# Define a gurobipy model for the decision problem
m = gp.Model('widgets')

In [5]:
path = 'https://raw.githubusercontent.com/Gurobi/modeling-examples/master/optimization101/Modeling_Session_1/'

In [12]:
data = pd.read_csv(path + 'cost.csv', index_col=[0,1])

In [13]:
transp_cost = data.squeeze()

In [14]:
transp_cost

production   distribution
Baltimore    Columbia        4.50
             Indianapolis    5.09
             Lexington       4.33
             Nashville       5.96
             Richmond        1.96
             St. Louis       7.30
Cleveland    Columbia        2.43
             Indianapolis    2.37
             Lexington       2.54
             Nashville       4.13
             Richmond        3.20
             St. Louis       4.88
Little Rock  Columbia        6.42
             Indianapolis    4.83
             Lexington       3.39
             Nashville       4.40
             Richmond        7.44
             St. Louis       2.92
Birmingham   Columbia        3.33
             Indianapolis    4.33
             Lexington       3.38
             Nashville       1.53
             Richmond        5.95
             St. Louis       4.01
Charleston   Columbia        3.02
             Indianapolis    2.61
             Lexington       1.61
             Nashville       4.44
             Richmond 

In [15]:
transp_cost.reset_index().pivot(index='production', columns='distribution', values='cost')

distribution,Columbia,Indianapolis,Lexington,Nashville,Richmond,St. Louis
production,,,,,,
Baltimore,4.50,5.09,4.33,5.96,1.96,7.30
Birmingham,3.33,4.33,3.38,1.53,5.95,4.01
Charleston,3.02,2.61,1.61,4.44,2.36,4.60
Cleveland,2.43,2.37,2.54,4.13,3.20,4.88
Little Rock,6.42,4.83,3.39,4.40,7.44,2.92


In [ ]:
max_prod = pd.Series([180,200,140,80,180], index = production, name = "max_production")
n_demand = pd.Series([89,95,121,101,116,181], index = distribution, name = "demand") 
max_prod.to_frame()
#n_demand.to_frame()

In [ ]:
frac = 0.75

In [ ]:
# loop through each p and d combination to create a decision variable
m = gp.Model('widgets')
x = {}
for p in production:
    for d in distribution:
        x[p,d] = m.addVar(name = p+"_to_"+d)
m.update()
x

In [ ]:
# Provide each set for the indices 
m = gp.Model('widgets')
x = m.addVars(production, distribution, name = 'prod_ship')
m.update()
x

In [ ]:
# The index of the tranporation costs have each combination of prodiction and distribution location
m = gp.Model('widgets')
x = m.addVars(transp_cost.index, name = 'prod_ship')
m.update()
x

In [ ]:
meet_demand = m.addConstrs((gp.quicksum(x[p,d] for p in production) >= n_demand[d] for d in distribution), name = 'meet_demand')
#meet_demand = m.addConstrs(x.sum('*', j) >= demand[j] for j in distribution)
m.update()
meet_demand

In [ ]:
can_produce = m.addConstrs((gp.quicksum(x[p,d] for d in distribution) <= max_prod[p] for p in production), name = 'can_produce')# (x.sum(i, '*')
must_produce = m.addConstrs((gp.quicksum(x[p,d] for d in distribution) >= frac*max_prod[p] for p in production), name = 'must_produce')
m.update()
can_produce

In [ ]:
m.setObjective(gp.quicksum(transp_cost[i,j]*x[i,j] for i in production for j in distribution), GRB.MINIMIZE) 

In [ ]:
m.write('widget_shipment.lp')

In [ ]:
m.optimize()

In [ ]:
x_values = pd.Series(m.getAttr('X', x), name = "shipment", index = transp_cost.index)
sol = pd.concat([transp_cost, x_values], axis=1)
#sol 
sol[sol.shipment > 0]

In [ ]:
# You can get the name and value of all the decision variables:
all_vars = {v.varName: v.x for v in m.getVars()} 
all_vars

In [ ]:
xvals = {k: v.x for k,v in x.items() if v.x > 0} 
xvals 

In [ ]:
x_values.sum(), n_demand.sum()

In [ ]:
# Sum the shipment amount by production facility
ship_out = sol.groupby('production')['shipment'].sum()
pd.DataFrame({'Remaining':max_prod - ship_out, 'Utilization':ship_out/max_prod})

In [ ]:
pd.DataFrame({'Remaining':[can_produce[p].Slack for p in production], 
              'Utilization':[1-can_produce[p].Slack/max_prod[p] for p in production]}, 
             index = production)

In [ ]:
# We use m2 for the second model
# These parts are the same as above outside of the new model name
m2 = gp.Model('widgets2')
x = m2.addVars(production, distribution, obj = transp_cost, name = 'prod_ship')
meet_demand = m2.addConstrs((gp.quicksum(x[p,d] for p in production) >= n_demand[d] for d in distribution), name = 'meet_demand')

In [ ]:
can_produce = m2.addConstrs((gp.quicksum(x[p,d] for d in distribution) <= max_prod[p] for p in production if p != 'Birmingham'), name = 'can_produce')
must_produce =  m2.addConstrs((gp.quicksum(x[p,d] for d in distribution) >= frac*max_prod[p] for p in production if p != 'Birmingham'), name = 'must_produce')

In [ ]:
xprod = m2.addVars(range(2), vtype = GRB.BINARY, obj = [50, 75], name = 'expand_Birmingham_prod')

In [ ]:
Birmingham_max = m2.addConstr(gp.quicksum(x['Birmingham',d] for d in distribution) <= max_prod['Birmingham'] + 25*xprod[0] + 50*xprod[1], name = 'Birmingham_add_max')
Birmingham_min = m2.addConstr(gp.quicksum(x['Birmingham',d] for d in distribution) >= frac*(max_prod['Birmingham'] + 25*xprod[0] + 50*xprod[1]), name = 'Birmingham_add_min')

In [ ]:
Birmingham_lim = m2.addConstr(gp.quicksum(xprod[i] for i in range(2)) <= 1, name = 'expansion_choice')

In [ ]:
m2.optimize()

In [ ]:
obj1 = m.getObjective()
obj2 = m2.getObjective()
print(f"The original model had a total cost of {round(obj1.getValue(),2)}")
print(f"The new formualtion has a total cost of {round(obj2.getValue(),2)}")

In [ ]:
pd.Series(m2.getAttr('X', xprod))

In [ ]:
x2_values = pd.Series(m2.getAttr('X', x), name = "shipment", index = transp_cost.index)
sol2 = pd.concat([transp_cost, x2_values], axis=1)
sol2[sol2.shipment > 0]

In [ ]:
m.dispose()
m2.dispose()
gp.disposeDefaultEnv()